# Sensitivity Analysis Demo

This notebook demonstrates how to perform sensitivity analysis on key economics parameters.

Sensitivity analysis helps answer questions like:
- How does NPV change with oil price?
- What's the impact of CAPEX variations?
- How sensitive are returns to production assumptions?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## Simple NPV Calculation

First, let's create a simplified NPV calculation function for demonstration.

In [ ]:
def calculate_simple_npv(
    qi: float,
    b: float,
    di: float,
    oil_price: float,
    capex: float,
    opex_per_month: float,
    nri: float = 0.75,
    months: int = 120,
    discount_rate: float = 0.10
) -> dict:
    """
    Calculate simple NPV for a single well.
    
    Returns:
    - dict with npv, eur, total_revenue, total_costs
    """
    # Convert to monthly
    di_monthly = 1 - (1 - di) ** (1/12)
    qi_monthly = qi * 30.4
    discount_monthly = discount_rate / 12
    
    # Calculate production profile
    production = []
    for t in range(1, months + 1):
        if b == 0:
            q_t = qi_monthly * np.exp(-di_monthly * t)
        else:
            q_t = qi_monthly / ((1 + b * di_monthly * t) ** (1/b))
        production.append(q_t)
    
    production = np.array(production)
    eur = production.sum()
    
    # Calculate cash flows
    revenue = production * oil_price * nri
    opex = np.full(months, opex_per_month)
    
    # CAPEX in month 0 (not discounted in this simple model)
    net_cash_flow = revenue - opex
    net_cash_flow[0] -= capex  # Subtract CAPEX in first month
    
    # Calculate NPV
    discount_factors = np.array([1 / (1 + discount_monthly) ** t for t in range(1, months + 1)])
    npv = np.sum(net_cash_flow * discount_factors)
    
    return {
        'npv': npv,
        'eur': eur,
        'total_revenue': revenue.sum(),
        'total_opex': opex.sum(),
        'capex': capex
    }

## Example 1: Single Variable Sensitivity (Oil Price)

In [ ]:
# Base case parameters
base_params = {
    'qi': 1000,
    'b': 1.2,
    'di': 0.50,
    'oil_price': 75,
    'capex': 8_000_000,
    'opex_per_month': 10_000,
    'nri': 0.75
}

# Range of oil prices to test
oil_prices = np.arange(40, 121, 5)
npvs = []

for price in oil_prices:
    result = calculate_simple_npv(**{**base_params, 'oil_price': price})
    npvs.append(result['npv'])

# Plot
plt.figure(figsize=(12, 6))
plt.plot(oil_prices, np.array(npvs) / 1e6, linewidth=2, marker='o')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
plt.axvline(x=base_params['oil_price'], color='green', linestyle='--', alpha=0.5, label='Base Case')
plt.title('Oil Price Sensitivity', fontsize=14)
plt.xlabel('Oil Price ($/bbl)', fontsize=12)
plt.ylabel('NPV10 ($MM)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Find breakeven price
breakeven_idx = np.argmin(np.abs(npvs))
breakeven_price = oil_prices[breakeven_idx]
print(f"Approximate breakeven oil price: ${breakeven_price:.0f}/bbl")
print(f"Base case NPV @ ${base_params['oil_price']}/bbl: ${npvs[np.where(oil_prices == base_params['oil_price'])[0][0]] / 1e6:.2f}MM")

## Example 2: Two-Variable Sensitivity Matrix (Oil Price vs CAPEX)

Create a heatmap showing how NPV varies with both oil price and CAPEX.

In [ ]:
# Define ranges
oil_prices = np.arange(50, 101, 10)
capex_values = np.arange(6_000_000, 10_500_000, 500_000)

# Calculate NPV matrix
npv_matrix = np.zeros((len(capex_values), len(oil_prices)))

for i, capex in enumerate(capex_values):
    for j, price in enumerate(oil_prices):
        result = calculate_simple_npv(**{**base_params, 'oil_price': price, 'capex': capex})
        npv_matrix[i, j] = result['npv'] / 1e6  # Convert to $MM

# Create heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(
    npv_matrix,
    annot=True,
    fmt='.1f',
    cmap='RdYlGn',
    center=0,
    xticklabels=[f'${p}' for p in oil_prices],
    yticklabels=[f'${c/1e6:.1f}MM' for c in capex_values],
    cbar_kws={'label': 'NPV10 ($MM)'}
)
plt.title('NPV Sensitivity: Oil Price vs CAPEX', fontsize=14)
plt.xlabel('Oil Price ($/bbl)', fontsize=12)
plt.ylabel('CAPEX', fontsize=12)
plt.tight_layout()
plt.show()

print(f"\nSensitivity Matrix Statistics:")
print(f"  Max NPV: ${npv_matrix.max():.2f}MM")
print(f"  Min NPV: ${npv_matrix.min():.2f}MM")
print(f"  Range: ${npv_matrix.max() - npv_matrix.min():.2f}MM")

## Example 3: Tornado Chart (Multiple Variables)

Compare the impact of different variables on NPV.

In [ ]:
# Calculate base case
base_result = calculate_simple_npv(**base_params)
base_npv = base_result['npv']

# Define sensitivities (±20% for each variable)
sensitivity_vars = {
    'Oil Price': ('oil_price', 0.8, 1.2),
    'CAPEX': ('capex', 0.8, 1.2),
    'Initial Prod (Qi)': ('qi', 0.8, 1.2),
    'Decline Rate (Di)': ('di', 0.8, 1.2),
    'OPEX': ('opex_per_month', 0.8, 1.2),
}

# Calculate impacts
impacts = []

for var_name, (param_name, low_mult, high_mult) in sensitivity_vars.items():
    # Low case
    low_params = base_params.copy()
    low_params[param_name] = base_params[param_name] * low_mult
    low_npv = calculate_simple_npv(**low_params)['npv']
    
    # High case
    high_params = base_params.copy()
    high_params[param_name] = base_params[param_name] * high_mult
    high_npv = calculate_simple_npv(**high_params)['npv']
    
    impacts.append({
        'Variable': var_name,
        'Low': (low_npv - base_npv) / 1e6,
        'High': (high_npv - base_npv) / 1e6,
        'Range': abs(high_npv - low_npv) / 1e6
    })

# Sort by range (total impact)
impacts_df = pd.DataFrame(impacts).sort_values('Range', ascending=True)

# Plot tornado chart
fig, ax = plt.subplots(figsize=(12, 8))
y_pos = np.arange(len(impacts_df))

ax.barh(y_pos, impacts_df['Low'], color='coral', alpha=0.7, label='-20%')
ax.barh(y_pos, impacts_df['High'], color='steelblue', alpha=0.7, label='+20%')
ax.axvline(x=0, color='black', linewidth=0.8)

ax.set_yticks(y_pos)
ax.set_yticklabels(impacts_df['Variable'])
ax.set_xlabel('Change in NPV10 ($MM)', fontsize=12)
ax.set_title('Tornado Chart: Sensitivity to ±20% Changes', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print(f"\nBase Case NPV: ${base_npv / 1e6:.2f}MM")
print(f"\nVariable Impacts (sorted by magnitude):")
for _, row in impacts_df.sort_values('Range', ascending=False).iterrows():
    print(f"  {row['Variable']:20s}: Range = ${row['Range']:.2f}MM")

## Example 4: Monte Carlo Simulation (Bonus)

Simulate NPV distribution with uncertain inputs.

In [ ]:
# Define parameter distributions (mean, std)
np.random.seed(42)
n_simulations = 1000

# Generate random samples
oil_prices_sim = np.random.normal(75, 15, n_simulations)  # Mean $75, StdDev $15
capex_sim = np.random.normal(8_000_000, 1_000_000, n_simulations)
qi_sim = np.random.normal(1000, 100, n_simulations)

# Calculate NPV for each simulation
npvs_sim = []

for i in range(n_simulations):
    params = base_params.copy()
    params['oil_price'] = max(oil_prices_sim[i], 20)  # Floor at $20
    params['capex'] = max(capex_sim[i], 5_000_000)  # Floor at $5MM
    params['qi'] = max(qi_sim[i], 500)  # Floor at 500 bbl/d
    
    result = calculate_simple_npv(**params)
    npvs_sim.append(result['npv'] / 1e6)

npvs_sim = np.array(npvs_sim)

# Plot distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
ax1.hist(npvs_sim, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
ax1.axvline(npvs_sim.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: ${npvs_sim.mean():.2f}MM')
ax1.axvline(np.percentile(npvs_sim, 10), color='orange', linestyle='--', linewidth=2, label=f'P10: ${np.percentile(npvs_sim, 10):.2f}MM')
ax1.axvline(np.percentile(npvs_sim, 90), color='green', linestyle='--', linewidth=2, label=f'P90: ${np.percentile(npvs_sim, 90):.2f}MM')
ax1.set_xlabel('NPV10 ($MM)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('NPV Distribution (Monte Carlo)', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Cumulative distribution
sorted_npvs = np.sort(npvs_sim)
cumulative = np.arange(1, len(sorted_npvs) + 1) / len(sorted_npvs) * 100
ax2.plot(sorted_npvs, cumulative, linewidth=2, color='steelblue')
ax2.axhline(50, color='red', linestyle='--', alpha=0.5, label='P50 (Median)')
ax2.axhline(10, color='orange', linestyle='--', alpha=0.5, label='P10')
ax2.axhline(90, color='green', linestyle='--', alpha=0.5, label='P90')
ax2.set_xlabel('NPV10 ($MM)', fontsize=12)
ax2.set_ylabel('Cumulative Probability (%)', fontsize=12)
ax2.set_title('Cumulative Distribution', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nMonte Carlo Results ({n_simulations} simulations):")
print(f"  Mean NPV: ${npvs_sim.mean():.2f}MM")
print(f"  Median (P50): ${np.median(npvs_sim):.2f}MM")
print(f"  Std Dev: ${npvs_sim.std():.2f}MM")
print(f"  P10: ${np.percentile(npvs_sim, 10):.2f}MM")
print(f"  P90: ${np.percentile(npvs_sim, 90):.2f}MM")
print(f"  Probability NPV > 0: {(npvs_sim > 0).sum() / n_simulations * 100:.1f}%")

## Next Steps

- Integrate with backend API endpoints for sensitivity matrices
- Add more sophisticated Monte Carlo with correlated variables
- Export results to CSV/Excel for further analysis
- Create dashboards with interactive widgets (ipywidgets)